In [ ]:
library(spant)
library(reshape2)
library(ggpubr)

# ggplot2 theme
theme_set(theme_bw())

# set random num generator seed
set.seed(1)

# simulate an ideal spectrum that looks like normal brain
ideal_spec <- sim_brain_1h(type = "normal_v1") %>% lb(5) %>% td2fd

# normalise the spectral intensity to NAA peak height
ideal_spec <- ideal_spec / as.numeric(peak_info(ideal_spec)$height)

# number of spectra to simulate
num_spec <- 512

# SNR range to investigate
SNRS <- c(2.5, 5.0, 7.5, 10.0, 15.0, 20.0, 25.0)

################################################################################
# Frequency drift test
################################################################################
force_sim1 = TRUE
if (!exists("sim1") | force_sim1) {
  sim1 <- TRUE
  runs <- length(SNRS)
  rats_freq_sd <- vector(mode = "numeric", length = runs)
  td_freq_sd <- vector(mode = "numeric", length = runs)
  rats_phase_sd <- vector(mode = "numeric", length = runs)
  td_phase_sd <- vector(mode = "numeric", length = runs)

  for (n in 1:runs) {
    # ideal unshifted data
    sim_data_0 <- ideal_spec %>% rep_dyn(num_spec) %>%
                  + sim_noise(1 / (SNRS[n]), dyns = num_spec)

    # define frequency adjustments for shift only example
    shifts <- seq(0, 10, length.out = num_spec)

    sim_data_1 <- sim_data_0 %>% shift(shifts, units = "hz")

    # correct using rats
    rats_res <- rats(sim_data_1, get_dyns(sim_data_1, 1))

    # correct using td method
    td_res <- tdsr(sim_data_1, get_dyns(sim_data_1, 1))

    # create a dataframe of true and estimated shift values for SNR = 5
    if (n == 2) {
      shifts_df_snr <- data.frame(shifts, RATS = as.numeric(rats_res$shifts),
                                  TDSR = as.numeric(td_res$shifts))

      shifts_df_snr <- melt(shifts_df_snr, "shifts", variable.name = "Method",
                            value.name = "estimate")

      shifts_df_snr$err <- shifts_df_snr$shifts - shifts_df_snr$estimate
    }

    rats_freq_sd[n] <- sd(shifts - rats_res$shifts)
    td_freq_sd[n]   <- sd(shifts - td_res$shifts)

    rats_phase_sd[n] <- sd(rats_res$phases)
    td_phase_sd[n]   <- sd(td_res$phases)
  }
}

freq_df <- data.frame(SNR = SNRS, RATS = rats_freq_sd, TDSR = td_freq_sd)
freq_df <- melt(freq_df, "SNR", variable.name = "Method", value.name = "error")

phase_df <- data.frame(SNR = SNRS, RATS = rats_phase_sd, TDSR = td_phase_sd)
phase_df <- melt(phase_df, "SNR", variable.name = "Method",
                 value.name = "error")

freq <- ggline(freq_df, "SNR", "error", color = "Method", palette = "jco",
       ylab = "Frequency error (Hz)", numeric.x.axis = T)

phase <- ggline(phase_df, "SNR", "error", color = "Method", palette = "jco",
       ylab = "Phase error (degrees)", numeric.x.axis = T)

lab_font <- list(size = 20, color = "black",
                 face = "bold", family = NULL)

shift_scatter <- ggscatter(shifts_df_snr, x = "shifts", y = "estimate",
                 col = "Method", ylab = "Estimated frequency shift (Hz)",
                 xlab = "True frequency shift (Hz)", palette = "jco",
                 shape = 1) + geom_abline(col = "black")

In [ ]:
ggarrange(freq, phase, shift_scatter, labels = "auto", font.label = lab_font,
          ncol = 3)

In [ ]:
################################################################################
# Frequency drift and random phase test
################################################################################
set.seed(2)
force_sim2 = TRUE
if (!exists("sim2") | force_sim2) {
  sim2 <- TRUE
  runs <- length(SNRS)
  rats_freq_sd <- vector(mode = "numeric", length = runs)
  td_freq_sd <- vector(mode = "numeric", length = runs)
  rats_phase_sd <- vector(mode = "numeric", length = runs)
  td_phase_sd <- vector(mode = "numeric", length = runs)

  for (n in 1:runs) {
    # ideal unshifted data
    sim_data_0 <- ideal_spec %>% rep_dyn(num_spec) %>%
      + sim_noise(1 / (SNRS[n]), dyns = num_spec)

    # define frequency adjustments for shift only example
    shifts <- seq(0, 10, length.out = num_spec)

    sim_data_1 <- sim_data_0 %>% shift(shifts, units = "hz")

    phases <- c(0, runif(num_spec - 1, -180, 180))
    sim_data_2 <- sim_data_1 %>% phase(phases)

    # correct using rats
    rats_res <- rats(sim_data_2, get_dyns(sim_data_2, 1))

    # correct using td method
    td_res <- tdsr(sim_data_2, get_dyns(sim_data_2, 1))

    rats_freq_sd[n]   <- sd(shifts - rats_res$shifts)
    td_freq_sd[n] <- sd(shifts - td_res$shifts)

    rats_phase_err <- phases - rats_res$phases

    # correct for multiples of 2pi
    rats_phase_err <- rats_phase_err - round(rats_phase_err / 180) * 180
    rats_phase_sd[n]   <- sd(rats_phase_err)

    td_phase_err <- phases - td_res$phases

    # correct for multiples of 2pi
    td_phase_err <- td_phase_err - round(td_phase_err / 180) * 180
    td_phase_sd[n] <- sd(td_phase_err)
  }
}

freq_df <- data.frame(SNR = SNRS, RATS = rats_freq_sd, TDSR = td_freq_sd)
freq_df <- melt(freq_df, "SNR", variable.name = "Method", value.name = "error")

phase_df <- data.frame(SNR = SNRS, RATS = rats_phase_sd, TDSR = td_phase_sd)
phase_df <- melt(phase_df, "SNR", variable.name = "Method",
                 value.name = "error")

freq <- ggline(freq_df, "SNR", "error", color = "Method", palette = "jco",
       ylab = "Frequency error (Hz)", numeric.x.axis = T)

phase <- ggline(phase_df, "SNR", "error", color = "Method", palette = "jco",
       ylab = "Phase error (degrees)", numeric.x.axis = T)

lab_font <- list(size = 20, color = "black",
                 face = "bold", family = NULL)

In [ ]:
ggarrange(freq, phase, labels = "auto", font.label = lab_font, ncol = 2)

In [ ]:
stackplot(get_dyns(sim_data_2, seq(from = 1, to = num_spec, by = 64)),
          xlim = c(4, 0.5), y_offset = 210)

In [ ]:
################################################################################
# Unstable residual water test
################################################################################

# simulate a water resonance
water_ideal <- sim_resonances(freq = 4.65, amp = 0.5, lw = 10)

set.seed(3)
water_phases <- c(0, runif(num_spec - 1, -180, 180))
water_sig <- water_ideal %>% rep_dyn(num_spec) %>% phase(water_phases)

# ideal unshifted data
SNR <- 15
sim_data_0 <- ideal_spec %>% rep_dyn(num_spec) %>%
              + sim_noise(1 / SNR, dyns = num_spec)

# define frequency adjustments for shift only example
shifts <- seq(0, 10, length.out = num_spec)

sim_data_1 <- sim_data_0 %>% shift(shifts, units = "hz")

set.seed(4)
phases <- c(0, runif(num_spec - 1, -180, 180))
sim_data_2 <- sim_data_1 %>% phase(phases)

sim_data_3 <- sim_data_2 + water_sig

# correct using rats
rats_res <- rats(sim_data_3, get_dyns(sim_data_3, 1))

# correct using td method
td_res <- tdsr(sim_data_3, get_dyns(sim_data_3, 1))

# create a dataframe of true and estimated shift values
shifts_df <- data.frame(shifts, RATS = as.numeric(rats_res$shifts),
                        TDSR = as.numeric(td_res$shifts))

shifts_df <- melt(shifts_df, "shifts", variable.name = "Method",
                  value.name = "estimate")

shifts_df$err <- shifts_df$shifts - shifts_df$estimate

shift_box <- ggboxplot(shifts_df, x = "Method", y = "err",
                       ylab = "True frequency shift - estimate (hz)") +
                       theme(plot.margin = unit(c(2,1,1,2), "lines"))

# create a dataframe of true and estimated phase values
phase_df <- data.frame(phases, RATS = as.numeric(rats_res$phases),
                       TDSR = as.numeric(td_res$phases))

phase_df <- melt(phase_df, "phases", variable.name = "Method",
                 value.name = "estimate")

phase_df$err <- phase_df$phases - phase_df$estimate
phase_df$err <- phase_df$err - round(phase_df$err / 180) * 180

phase_box <- ggboxplot(phase_df, x = "Method", y = "err",
                       ylab = "True phase - estimate (degrees)") +
                       theme(plot.margin = unit(c(2,1,1,2), "lines"))

In [ ]:
ggarrange(shift_box, phase_box, labels = "auto", font.label = lab_font)

In [ ]:
lw <- 0.35
subset <- seq(from = 1, to = 512, length.out = 32)
par(mfrow = c(1, 3))
stackplot(get_dyns(sim_data_3, subset), xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("a", at = 4, padj = 0.5, font = 2)

stackplot(get_dyns(td_res$corrected, subset), xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("b", at = 4, padj = 0.5, font = 2)

stackplot(get_dyns(rats_res$corrected, subset), xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("c", at = 4, padj = 0.5, font = 2)

In [ ]:
# moderate baseline issue

num_spec <- 32

# simulate a water resonance
water_ideal <- sim_resonances(freq = 4.65, amp = 0.2, lw = 10)

set.seed(3)
water_phases <- c(0, runif(num_spec - 1, -180, 180))
water_sig <- water_ideal %>% rep_dyn(num_spec) %>% phase(water_phases)

# ideal unshifted data
SNR <- 15
sim_data_0 <- ideal_spec %>% rep_dyn(num_spec) %>%
              + sim_noise(1 / SNR, dyns = num_spec)

# define frequency adjustments for shift only example
shifts <- seq(0, 10, length.out = num_spec)

sim_data_1 <- sim_data_0 %>% shift(shifts, units = "hz")

set.seed(4)
phases <- c(0, runif(num_spec - 1, -180, 180))
sim_data_2 <- sim_data_1 %>% phase(phases)

sim_data_3 <- sim_data_2 + water_sig

# correct using rats
rats_res <- rats(sim_data_3, get_dyns(sim_data_3, 1))

# correct using td method
td_res <- tdsr(sim_data_3, get_dyns(sim_data_3, 1))

# create a dataframe of true and estimated shift values
shifts_df <- data.frame(shifts, RATS = as.numeric(rats_res$shifts),
                        TDSR = as.numeric(td_res$shifts))

shifts_df <- melt(shifts_df, "shifts", variable.name = "Method",
                  value.name = "estimate")

shifts_df$err <- shifts_df$shifts - shifts_df$estimate

shift_box <- ggboxplot(shifts_df, x = "Method", y = "err",
                 ylab = "True frequency shift - estimate (hz)") +
                 theme(plot.margin = unit(c(2,1,1,2), "lines"))

# create a dataframe of true and estimated phase values
phase_df <- data.frame(phases, RATS = as.numeric(rats_res$phases),
                 TDSR = as.numeric(td_res$phases))

phase_df <- melt(phase_df, "phases", variable.name = "Method",
                  value.name = "estimate")

phase_df$err <- phase_df$phases - phase_df$estimate
phase_df$err <- phase_df$err - round(phase_df$err / 180) * 180

phase_box <- ggboxplot(phase_df, x = "Method", y = "err",
                 ylab = "True phase - estimate (degrees)") +
                 theme(plot.margin = unit(c(2,1,1,2), "lines"))

lw <- 0.35

In [ ]:
par(mfrow = c(1, 3))
stackplot(sim_data_3, xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("a", at = 4, padj = 0.5, font = 2)

stackplot(td_res$corrected, xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("b", at = 4, padj = 0.5, font = 2)

stackplot(rats_res$corrected, xlim = c(4, 0.5), y_offset = 0,
          restore_def_par = FALSE, lwd = lw)

mtext("c", at = 4, padj = 0.5, font = 2)